# Subset of data for HEROKU

In [2]:
PROC_DATA_PREFIX = '/Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES'

In [8]:
# Count sentence lengths
def count_sent_len(data_prefix, file_in, file_out):
    with open(data_prefix + '/' + file_in, 'r', encoding='utf8') as inF, \
        open(data_prefix + '/' + file_out, 'w', encoding='utf8') as toF_nums:
        for line in inF:
            line_list = line.split('\t')
            src_str, trg_str = line_list[0].strip(), line_list[1].strip()
            src_len, trg_len = len(src_str.split()), len(trg_str.split())
            toF_nums.write('{}\t{}\n'.format(src_len, trg_len))

In [9]:
# heroku limits dev db size to 10,000 rows total
# Process:
# Get 2 files - 1,000 line long raw text and 1,000 line long tokenized text file
# Extract a lexicon from the tokenized text file
# Iterate over glossary and the lexicon to keep only those terms which occur in the tokenized file
# with the aim of getting a 9,000 line glossary file
# Remove too long sentences from the raw text file - 

import pandas as pd

count_sent_len(PROC_DATA_PREFIX, 
               'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_40',  # input file
               'sent_lengths_40.txt'                              # output file
              )

my_data = pd.read_csv(PROC_DATA_PREFIX + '/' + 'sent_lengths_40.txt', sep='\t', header=None)
my_data.columns = ['src_len', 'trg_len']
my_data.head()

,src_len,trg_len
0,7,7
1,4,4
2,4,4
3,5,5
4,5,5


In [ ]:
# Keep only those line which are less than the threshold number of words
threshold = 20  # words
with open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_40', 'r', encoding='utf8') as inF, \
     open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20', 'w', encoding='utf8') as toF:
    for idx, line in enumerate(inF):
        if (my_data.loc[:, 'src_len'][idx] < threshold) and (my_data.loc[:, 'trg_len'][idx] < threshold):
            toF.write(line)

!wc -l {PROC_DATA_PREFIX}/'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_40'
!wc -l {PROC_DATA_PREFIX}/'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20'

 14561 /Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES/en_ru_sort_uniq_nearduplrem_dissim_tok_2std_40
    3398 /Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES/en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20

In [ ]:
# raw text file
threshold = 20
with open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_2std_40', 'r', encoding='utf8') as inF, \
     open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_2std_20', 'w', encoding='utf8') as toF:
    for idx, line in enumerate(inF):
        if (my_data.loc[:, 'src_len'][idx] < threshold) and (my_data.loc[:, 'trg_len'][idx] < threshold):
            toF.write(line)

!wc -l {PROC_DATA_PREFIX}/'en_ru_sort_uniq_nearduplrem_dissim_2std_40'
!wc -l {PROC_DATA_PREFIX}/'en_ru_sort_uniq_nearduplrem_dissim_2std_20'

  14561 /Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES/en_ru_sort_uniq_nearduplrem_dissim_2std_40
    3398 /Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES/en_ru_sort_uniq_nearduplrem_dissim_2std_20

In [ ]:
one_word_sent_idx = []
with open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20', 'r', encoding='utf8') as inF:
    for idx, line in enumerate(inF):
        src, trg = line.split('\t')
        src, trg = src.strip(), trg.strip()
        if (len(src.split()) == 1) or (len(trg.split()) == 1):
            one_word_sent_idx.append(idx)
            
print(len(one_word_sent_idx))
print(one_word_sent_idx)

In [ ]:
# Remove single word lines
with open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20', 'r', encoding='utf8') as inF, \
    open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20_oneword', 'w', encoding='utf8') as toF:
    for idx, line in enumerate(inF):
        if idx not in one_word_sent_idx:
            toF.write(line)

!wc -l {PROC_DATA_PREFIX}/en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20_oneword

with open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_2std_20', 'r', encoding='utf8') as inF, \
    open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_2std_20_oneword', 'w', encoding='utf8') as toF:
    for idx, line in enumerate(inF):
        if idx not in one_word_sent_idx:
            toF.write(line)

!wc -l {PROC_DATA_PREFIX}/en_ru_sort_uniq_nearduplrem_dissim_2std_20_oneword

In [ ]:
# get rid of near duplicate lines at a lower cutoff point compared to the main corpus (which was 0.85)
threshold = 0.66
with open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20_oneword', 'r', encoding='utf8') as inF, \
     open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20_oneword_1000', 'w', encoding='utf8') as toF:
    prev_line = inF.readline()
    for curr_line in inF:
        if jaro_similarity(prev_line, curr_line) > threshold:  # Two lines are very similar, forget previous line
            prev_line = curr_line
        else:
            toF.write(prev_line)   # If not similar, put previous line into file,
            prev_line = curr_line  # then forget it

In [6]:
!wc -l {PROC_DATA_PREFIX}/'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20_oneword_1000'

     980 /Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES/en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20_oneword_1000


 3398 /Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES/en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20
    2844 /Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES/en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20_short

In [ ]:
# Do approximately the same for raw text file
threshold = 0.65
with open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_2std_20_oneword', 'r', encoding='utf8') as inF, \
     open(PROC_DATA_PREFIX + '/' + 'en_ru_sort_uniq_nearduplrem_dissim_2std_20_oneword_1000', 'w', encoding='utf8') as toF:
    prev_line = inF.readline()
    for curr_line in inF:
        if jaro_similarity(prev_line, curr_line) > threshold:  # Two lines are very similar, forget previous line
            prev_line = curr_line
        else:
            toF.write(prev_line)   # If not similar, put previous line into file,
            prev_line = curr_line  # then forget it

!wc -l {PROC_DATA_PREFIX}/'en_ru_sort_uniq_nearduplrem_dissim_2std_20_oneword_1000'

In [ ]:
!cp {PROC_DATA_PREFIX}/'en_ru_sort_uniq_nearduplrem_dissim_2std_20_oneword_1000' \
    {PROC_DATA_PREFIX}/'en_ru_heroku_1000'

!cp {PROC_DATA_PREFIX}/'en_ru_sort_uniq_nearduplrem_dissim_tok_2std_20_oneword_1000' \
    {PROC_DATA_PREFIX}/'en_ru_heroku_tok_1000'

In [5]:
!wc -l {PROC_DATA_PREFIX}/'en_ru_heroku_1000'
!wc -l {PROC_DATA_PREFIX}/'en_ru_heroku_tok_1000'

     994 /Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES/en_ru_heroku_1000
     980 /Users/alexskrn/Documents/TEXT_CORPORA/united_nations_corpus/en-ru/PREPROC_FILES/en_ru_heroku_tok_1000


## Examples of jaro similarity scores

In [ ]:
jaro_similarity('(a) Protection of civilians	a) Защита гражданского населения', 
                '(a) Protection of civilians	а) Защита гражданского населения')
# 0.9210922787193973

In [ ]:
t1 = 'Welcoming the report of the Secretary-General of 7 December 1999 (S/1999/1219) and the observations and recommendations contained therein,	с удовлетворением отмечая доклад Генерального секретаря от 7 декабря 1999 года (S/1999/1219) и содержащиеся в нем замечания и рекомендации,'
t2 = 'Welcoming the report of the Secretary-General of 7 January 2013 (S/2013/07) on the United Nations operation in Cyprus,	приветствуя доклад Генерального секретаря от 7 января 2013 года (S/2013/7) об операции Организации Объединенных Наций на Кипре,'
jaro_similarity(t1, t2)
# 0.738505867179379

In [ ]:
t1 = 'benefit from same terms conditions service mutatis mutandis as permanent judges international tribunal for rwanda	а пользуются mutatis mutandis теми же условиями службы что постоянные судьи международного трибунала по руанде'
t2 = 'benefit from same terms conditions service mutatis mutandis as permanent judges international tribunal	а пользуются mutatis mutandis теми же условиями службы что постоянные судьи международного трибунала'
jaro_similarity(t1, t2)
# 0.8932163382594417

In [ ]:
t1 = 'impellers	крыльчатки'
t2 = 'implementation	осуществление'
t3 = 'implementation mechanism	механизм осуществления'
print(jaro_similarity(t1, t2))
print(jaro_similarity(t2, t3))
# 0.5523809523809524
# 0.7938196555217831